## Objetivo del taller

-   Consultar, descargar y **cargar en Python** imágenes Sentinel (S2
    óptico y S1 radar).
-   Crear **composites** (RGB), índices (p. ej. **NDVI**), **enmascarar
    nubes**, recortar por AOI y **exportar GeoTIFF**.
-   Dos rutas de trabajo:
    1.  **Flujo A (recomendado):** STAC de Microsoft Planetary Computer
        (S2-L2A y S1-RTC).  
    2.  **Flujo B:** lectura **local** de productos `.SAFE`
        (carpetas/JP2/GeoTIFF) con `rasterio`/`rioxarray`.

------------------------------------------------------------------------

## Requisitos e instalación (Conda)

``` bash
# crear entorno (opcional)
conda create -n sentinel_env python=3.11 -y
conda activate sentinel_env

# paquetes base
pip install rasterio rioxarray xarray geopandas shapely
pip install matplotlib numpy tqdm

# acceso vía STAC (Planetary Computer)
pip install pystac-client odc-stac planetary-computer stackstac

# (opcional) filtros y utilidades
pip install scipy scikit-image
```

> Nota: Para usar Planetary Computer, muchas colecciones requieren
> **firma de URLs** con `planetary_computer`. El registro de cuenta es
> gratuito.

------------------------------------------------------------------------

## Configurar el área de interés (AOI)

Defina su **caja** (lon/lat) o cargue un **GeoJSON**. Ajuste estas
variables:

``` python
# Coordenadas aproximadas de Aguascalientes (lon_min, lat_min, lon_max, lat_max)
bbox = [-102.5, 21.6, -101.8, 22.2]

# Intervalo temporal (ISO 8601)
timerange = "2024-11-01/2024-12-15"
max_cloud = 20  # % nubes (para Sentinel-2)
```

------------------------------------------------------------------------

## Flujo A · Sentinel-2 L2A (Planetary Computer, STAC)

### 1) Buscar escenas con STAC

``` python
from pystac_client import Client
import planetary_computer as pc

stac = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime=timerange,
    query={"eo:cloud_cover": {"lt": max_cloud}},
    max_items=5,
)

items = list(search.get_items())
len(items), items[0].id
```

------------------------------------------------------------------------

### 2) Cargar bandas con *stackstac* → `xarray`

``` python
import stackstac, xarray as xr

item = pc.sign(items[0])  # firma de URLs
assets = ["B04", "B03", "B02", "B08", "SCL"]  # Rojo, Verde, Azul, NIR y clasificación

da = stackstac.stack(
    item,
    assets=assets,
    chunksize=2048,      # dask-chunks (ajuste a su RAM)
    epsg=None,           # respeta la proyección nativa
    dtype="uint16",
)

# Convierte a Dataset por banda
ds = da.to_dataset(dim="band")  # ds["B04"], ds["B03"], ...
ds
```

> *Tip:* Los valores de reflectancia de S2-L2A suelen venir escalados
> (0–10000). Para visualizar, divida entre **10000** y recorte a
> \[0,1\].

------------------------------------------------------------------------

### 3) Compuesto RGB verdadero (B4-B3-B2) y falso color NIR (B8-B4-B3)

``` python
import numpy as np

def stretch01(arr, low=0, high=10000):
    arr = arr.clip(low, high)
    return ((arr - low) / (high - low)).clip(0, 1)

rgb_true = xr.concat([ds["B04"], ds["B03"], ds["B02"]], dim="band").transpose("band", "y", "x")
rgb_true_viz = stretch01(rgb_true)

rgb_false = xr.concat([ds["B08"], ds["B04"], ds["B03"]], dim="band").transpose("band", "y", "x")
rgb_false_viz = stretch01(rgb_false)
```

``` python
import matplotlib.pyplot as plt

plt.figure(figsize=(7,7))
plt.imshow(np.moveaxis(rgb_true_viz.values, 0, -1))
plt.title("Sentinel-2 L2A · Compuesto RGB verdadero (B4-B3-B2)")
plt.axis("off")
plt.show()

plt.figure(figsize=(7,7))
plt.imshow(np.moveaxis(rgb_false_viz.values, 0, -1))
plt.title("Sentinel-2 L2A · Falso color NIR (B8-B4-B3)")
plt.axis("off")
plt.show()
```

------------------------------------------------------------------------

### 4) NDVI y **máscara de nubes** (SCL o QA60)

``` python
nir = ds["B08"].astype("float32")
red = ds["B04"].astype("float32")
ndvi = (nir - red) / (nir + red)
ndvi = ndvi.clip(-1, 1)
ndvi.name = "NDVI"

# SCL: Scene Classification Layer (L2A). Usamos 3 categorías de nube/sombra
# 3 = Shadow, 8 = Medium probability cloud, 9 = High probability cloud, 10 = Thin cirrus, 11 = Snow
scl = ds["SCL"]
cloud_mask = scl.isin([3, 8, 9, 10])

ndvi_clear = ndvi.where(~cloud_mask)
ndvi, ndvi_clear
```

``` python
plt.figure(figsize=(7,6))
im = plt.imshow(ndvi_clear, vmin=-0.2, vmax=0.8)
plt.colorbar(im, fraction=0.046, pad=0.04, label="NDVI (nubes enmascaradas)")
plt.title("NDVI con máscara de nubes (SCL)")
plt.axis("off")
plt.show()
```

------------------------------------------------------------------------

### 5) Recorte por AOI (GeoJSON/Shapefile) y exportar GeoTIFF

``` python
import geopandas as gpd
import rioxarray

# Cargue su AOI, por ejemplo "aoi.geojson"
# aoi = gpd.read_file("aoi.geojson")
# Para demo, creamos un rectángulo desde bbox
from shapely.geometry import box
aoi = gpd.GeoDataFrame(geometry=[box(*bbox)], crs="EPSG:4326")

# Reproyectar y recortar
aoi_proj = aoi.to_crs(ndvi.rio.crs)  # stackstac provee crs/espg
ndvi_clip = ndvi_clear.rio.clip(aoi_proj.geometry, aoi_proj.crs, drop=True)

# Exportar
ndvi_clip.rio.to_raster("ndvi_clip.tif", compress="deflate")
"Archivo escrito: ndvi_clip.tif"
```

------------------------------------------------------------------------

## Flujo A · Sentinel-1 RTC (Radar, VV/VH)

El conjunto **sentinel-1-rtc** en Planetary Computer ya viene
**radiométrica y geométricamente corregido (RTC)**. Así evitamos pasos
complejos (calibración/terreno) típicos de S1-GRD crudo.

``` python
search_s1 = stac.search(
    collections=["sentinel-1-rtc"],
    bbox=bbox,
    datetime=timerange,
    query={"sar:polarizations": {"in": ["VV","VH"]}},
    max_items=2,
)
items_s1 = list(search_s1.get_items())
item_s1 = pc.sign(items_s1[0])

da_s1 = stackstac.stack(item_s1, assets=["VV","VH"], chunksize=2048, dtype="float32")
ds_s1 = da_s1.to_dataset("band")  # ds_s1["VV"], ds_s1["VH"]
ds_s1
```

``` python
vv = ds_s1["VV"]
# dB para visualización
vv_db = 10 * np.log10(vv.where(vv > 0))
plt.figure(figsize=(7,6))
plt.imshow(vv_db, vmin=-20, vmax=5)
plt.title("Sentinel-1 RTC · VV (dB)")
plt.axis("off")
plt.show()
```

> *Opcional:* filtro de moteado (**speckle**) con mediana (ventana 3–5
> px) usando `scipy.ndimage.median_filter`.

------------------------------------------------------------------------

## Flujo B · Lectura **local** de `.SAFE` (S2 L1C/L2A)

Estructura típica:
`S2A_MSIL2A_... .SAFE/GRANULE/.../IMG_DATA/.../B04.jp2`

``` python
import rasterio as rio
from glob import glob

safe_root = "/ruta/a/S2_MSIL2A_...SAFE"
b04_path = glob(f"{safe_root}/GRANULE/*/IMG_DATA/*_B04*.jp2")[0]
b03_path = glob(f"{safe_root}/GRANULE/*/IMG_DATA/*_B03*.jp2")[0]
b02_path = glob(f"{safe_root}/GRANULE/*/IMG_DATA/*_B02*.jp2")[0]

with rio.open(b04_path) as r4, rio.open(b03_path) as r3, rio.open(b02_path) as r2:
    b4 = r4.read(1)
    b3 = r3.read(1)
    b2 = r2.read(1)
    profile = r4.profile

rgb_local = np.stack([b4, b3, b2], axis=0)
rgb_local_viz = stretch01(rgb_local)

plt.figure(figsize=(7,7))
plt.imshow(np.moveaxis(rgb_local_viz, 0, -1))
plt.title("S2 local · Compuesto RGB verdadero (B4-B3-B2)")
plt.axis("off")
plt.show()
```

> Para **L1C**, puede usarse `QA60` para nubes. Para **L2A**, emplee
> `SCL`. Ubique el archivo correspondiente en `IMG_DATA` o subcarpetas
> `R10m/R20m/R60m`.

------------------------------------------------------------------------

## Descarga masiva (opcional) con `pystac-client` + `requests`

``` python
# Guardar activos (bandas) a disco desde un item firmado
import requests, os

def download_assets(item, assets=("B04","B03","B02","B08","SCL"), outdir="S2_download"):
    os.makedirs(outdir, exist_ok=True)
    signed = pc.sign(item)
    for a in assets:
        href = signed.assets[a].href
        fname = os.path.join(outdir, f"{signed.id}_{a}.tif")
        with requests.get(href, stream=True) as r:
            r.raise_for_status()
            with open(fname, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk: f.write(chunk)
    return outdir

# out = download_assets(items[0])
# out
```

------------------------------------------------------------------------

## Recorte por shapefile y reproyección

``` python
# Suponga un shapefile "limite_municipal.shp"
aoi = gpd.read_file("limite_municipal.shp")
aoi = aoi.to_crs(ndvi.rio.crs)

ndvi_mun = ndvi.rio.clip(aoi.geometry, aoi.crs, drop=True)
ndvi_mun.rio.to_raster("ndvi_municipio.tif", compress="deflate")
```

------------------------------------------------------------------------

## Compuesto temporal (mediana) · Sentinel-2

``` python
# Apilar varias escenas y calcular mediana libre de nubes
items_many = [pc.sign(it) for it in items[:4]]

stacks = []
for it in items_many:
    da_i = stackstac.stack(it, assets=["B04","B03","B02","B08","SCL"], chunksize=1024, dtype="uint16")
    ds_i = da_i.to_dataset("band")
    clouds = ds_i["SCL"].isin([3,8,9,10])
    nir_i, red_i = ds_i["B08"].astype("float32"), ds_i["B04"].astype("float32")
    ndvi_i = ((nir_i - red_i) / (nir_i + red_i)).where(~clouds)
    stacks.append(ndvi_i)

ndvi_stack = xr.concat(stacks, dim="time")
ndvi_median = ndvi_stack.median("time", skipna=True)
ndvi_median.rio.to_raster("ndvi_median.tif", compress="deflate")
ndvi_median
```

``` python
plt.figure(figsize=(7,6))
im = plt.imshow(ndvi_median, vmin=0.0, vmax=0.8)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title("NDVI · mediana temporal (cloud-free)")
plt.axis("off")
plt.show()
```

------------------------------------------------------------------------

## Consejos prácticos

-   Trabaje con **L2A** (reflectancia de superficie) cuando sea posible.
-   Para radar, use **S1-RTC** (corrigido) si no domina preprocesos GRD.
-   Controle memoria con **chunks** (Dask) y recortes por AOI.
-   Documente todo en **Quarto/Jupyter**, con nombres de archivo
    reproducibles.
-   Revise metadatos (CRS, resolución) antes de exportar o combinar
    capas.

------------------------------------------------------------------------

## Referencias rápidas

-   Catálogo STAC de **Planetary Computer**:
    `https://planetarycomputer.microsoft.com/catalog`
-   Colecciones: `sentinel-2-l2a`, `sentinel-1-rtc`
-   Paquetes: `pystac-client`, `planetary-computer`, `stackstac`,
    `odc-stac`, `rioxarray`, `rasterio`

**¡Listo!** Esta presentación incluye ambos flujos para que puedas
adaptar el proceso a tu infraestructura (online/offline).